In [1]:
import openai
openai.api_key = "***"
import json
import time
import os
from json import JSONDecodeError
data = json.load(open('./vl_checklist.json')) ## Json file for sampled 2000 vl-checklist data
SAVE_MATCHING_FILE = "../vl-checklist-matching-json/{}_{}.json"
CAPTION_PATH = "../vl-checklist-caption-json/{}.json"
TEXT_PATH = "../vl-checklist-text-json/{}_{}.json"

In [2]:
def get_dense_caption(id):
    f = open(CAPTION_PATH.format(id))
    result = list(json.load(f).keys())
    return result
def get_sentence_json(row_id, pos_or_neg):
    f = open(TEXT_PATH.format(row_id, pos_or_neg))
    try:
        result = json.load(f)
        if type(result) == dict:
            return list(result["objects"].keys())
        else:
            return list(json.loads(result)["objects"].keys())
    except Exception as e:
        return list(json.loads(result)["objects"].keys())

In [15]:
def match_objects(sentence, object, list):
    try:
        input = "Given a list of labels in triple quotes \"\"\"{}\"\"\" ".format(list) + \
                "and a sentence in triple quotes \"\"\"{}\"\"\". Tell me which labels in the list could refer to ".format(sentence) + \
                "the \"{}\" from the sentence. Only output the results in json format with 1 key named labels. If none of the labels refer to the \"{}\", set the value in the json to be an empty list".format(object, object)
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": input}], temperature=0)
        return json.loads(completion["choices"][0]["message"]["content"])
    except Exception as e:
        raise e
def generate_caption_to_image(caption, obejct_list, image_list):
    result = {}
    for object in obejct_list:
        matched = match_objects(caption, object, image_list)["labels"]
        result[object] = matched
    return result

In [ ]:
for idx, i in enumerate(data):
    if not (os.path.exists(SAVE_MATCHING_FILE.format(idx, "pos").format(idx)) and os.path.exists(SAVE_MATCHING_FILE.format(idx, "neg"))):
        try:
            print(idx)
            image_list = get_dense_caption(idx)
            pos_object_list = get_sentence_json(idx, 'pos')
            neg_object_list = get_sentence_json(idx, 'neg')
            pos_match =  generate_caption_to_image(i[1]["POS"][0], pos_object_list, image_list)
            with open(SAVE_MATCHING_FILE.format(idx, "pos"), 'w') as f:
                json.dump(pos_match, f)
            neg_match =  generate_caption_to_image(i[1]["NEG"][0], neg_object_list, image_list)
            with open(SAVE_MATCHING_FILE.format(idx, "neg"), 'w') as f:
                json.dump(neg_match, f)
        except JSONDecodeError as e:
            print(e)
        except FileNotFoundError as e:
            print(e)
